In [1]:
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
"""
Global values
"""
FEATURE_LEN = 5
folder = 'boxoffice_10'


In [3]:

user_rating = pickle.load(open('../data/'+folder+'/user_rating.pkl', 'rb'))

user_rating_valid = pickle.load(open('../data/'+folder+'/user_rating_validation.pkl', 'rb'))

user_rating_test = pickle.load(open('../data/'+folder+'/user_rating_test.pkl', 'rb'))

df_train = []
max_user = 0
max_movie = 0
for user in user_rating:
    user_i = int(user)
    for tup in user_rating[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_train.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i
        
df_test = []
for user in user_rating_test:
    user_i = int(user)
    for tup in user_rating_test[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_test.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i
        

df_valid = []
for user in user_rating_valid:
    user_i = int(user)
    for tup in user_rating_valid[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_valid.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i

#user_s, movie_s, rating_s = shuffle(user_list, movie_list, rating_list)

#df_train, df_test = train_test_split(df, test_size = 0.2)

"""
"""
user_s = [t[0] for t in df_train]
movie_s = [t[1] for t in df_train]
rating_s = [t[2] for t in df_train]

user_s_test = [t[0] for t in df_test]
movie_s_test = [t[1] for t in df_test]
rating_s_test = [t[2] for t in df_test]

user_s_valid = [t[0] for t in df_valid]
movie_s_valid = [t[1] for t in df_valid]
rating_s_valid = [t[2] for t in df_valid]

print("training number: ", len(user_s))
print("validaton number: ", len(user_s_valid))
print("test number: ", len(user_s_test))

training number:  79460
validaton number:  9933
test number:  9933


In [4]:
user_indecies_train = user_s
item_indecies_train = movie_s
rates_train = rating_s

user_indecies_test = user_s_test
item_indecies_test = movie_s_test
rates_test = rating_s_test

user_indecies_valid = user_s_valid
item_indecies_valid = movie_s_valid
rates_valid = rating_s_valid


In [5]:
max_movie

206

In [6]:
totle_len = len(user_s)
totle_len

79460

In [7]:
def get_bias(user, movie, rating):
    print('max user:', max_user)
    print('max movie:', max_movie)
    average = np.mean(rating)
    user_dict = {}
    movie_dict = {}
    for i in range(len(user)):
        user_dict[user[i]] = [rating[i]] if user[i] not in user_dict else user_dict[user[i]] + [rating[i]]
        movie_dict[movie[i]] = [rating[i]] if movie[i] not in movie_dict else movie_dict[movie[i]] + [rating[i]]
    
    user_bias = [0 for i in range(max_user + 1)]
    movie_bias = [0 for i in range(max_movie + 1)]
    print(user_dict[2253])
    print(len(user_bias), len(movie_bias))
    for key in user_dict:
        user_bias[int(key)] = np.mean(user_dict[key]) - average
    for key in movie_dict:
        movie_bias[int(key)] = np.mean(movie_dict[key]) - average
    
    return user_bias, movie_bias, average

user_bias, movie_bias, average_score = get_bias(user_s, movie_s, rating_s)


    

max user: 3549
max movie: 206
[2.0, 3.0, 4.0, 4.0, 3.5, 4.0, 3.0, 3.5, 2.0, 3.5]
3550 207


In [8]:
user_bias_r = np.reshape(user_bias,[-1,1])
movie_bias_r = np.reshape(movie_bias,[1,-1])

In [9]:
U = tf.Variable(initial_value=tf.truncated_normal([max_user+1, FEATURE_LEN], stddev=0.2, mean=0), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, max_movie+1], stddev=0.2, mean=0), name='movies')

In [10]:

U_plus_bias = tf.concat([U, tf.constant(user_bias_r, dtype=tf.float32, name="user_bias"), tf.ones((max_user+1,1), dtype=tf.float32, name="user_bias_ones")], 1)
P_plus_bias = tf.concat([P, tf.ones((1, max_movie+1), name="movie_bias_ones", dtype=tf.float32), tf.constant(movie_bias_r, dtype=tf.float32, name="movie_bias")], 0)


In [11]:
#result = tf.matmul(U_plus_bias, P_plus_bias)
result = tf.matmul(U, P)

In [12]:
result_flatten = tf.reshape(result, [-1])
R = tf.gather(result_flatten, user_s * tf.shape(result)[1] + movie_s, name='extracting_user_rate')

In [13]:
diff_op = tf.subtract(R, rating_s, name='trainig_diff')
diff_op_squared = tf.nn.l2_loss(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

In [14]:
lda = tf.constant(0.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.nn.l2_loss(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.nn.l2_loss(P, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [15]:
cost = tf.add(base_cost, regularizer)
#cost = base_cost

In [16]:
lr = tf.constant(.0001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.98, staircase=True)
#learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [17]:


# accuracy
R_test = tf.gather(result_flatten, user_indecies_train * tf.shape(result)[1] + item_indecies_train, name='extracting_user_rate_test')
#R_test = tf.cast(R_test, tf.float64)
rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_train, R_test))))
#print(sess.run(rmse))


In [18]:
for i in user_indecies_train:
    if i < 0:
        print(i)
        break

In [19]:


# accuracy
test_R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + item_indecies_test, name='extracting_user_rate_test')
#test_R_test = tf.cast(test_R_test, tf.float64)
test_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_test, test_R_test))))
#print(sess.run(rmse))



In [20]:
# accuracy
valid_R_test = tf.gather(result_flatten, user_indecies_valid * tf.shape(result)[1] + item_indecies_valid, name='extracting_user_rate_valid')
#test_R_test = tf.cast(test_R_test, tf.float64)
valid_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_valid, valid_R_test))))
#print(sess.run(rmse))

In [21]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
minVR = 999
not_change_n = 0
for i in range(10001):
    _, c, r, tr, vr = sess.run([training_step, cost, rmse, test_rmse, valid_rmse])
    if i%20 == 0:
        print(i, c, r, vr, tr, not_change_n)
    
        if vr + 0.001 < minVR:
            minVR = vr
            not_change_n = 0
        else:
            not_change_n += 1

        if not_change_n > 10:
            break
            
    
    """
    if i%500 == 0:
        r, tr = sess.run([rmse, test_rmse])
        #r = sess.run([rmse])
        print("rmse:", i, r, tr)
    """
    
    

0 520576.0 3.61978 3.63133 3.60875 0
20 519902.0 3.61744 3.63115 3.60857 0
40 518139.0 3.6113 3.62749 3.60478 1
60 507863.0 3.57531 3.59594 3.5726 0
80 444549.0 3.34503 3.3801 3.35401 0
100 250998.0 2.51348 2.58383 2.55525 0
120 111366.0 1.67423 1.75803 1.74078 0
140 62032.1 1.24952 1.3287 1.31871 0
160 42301.7 1.03183 1.1033 1.09657 0
180 33243.8 0.914703 0.979419 0.974872 0
200 28578.4 0.848087 0.907945 0.905227 0
220 25928.7 0.807811 0.864484 0.863355 0
240 24292.8 0.781907 0.836618 0.836856 0
260 23209.5 0.764272 0.817842 0.819232 0
280 22450.0 0.751662 0.804628 0.806978 0
300 21892.4 0.742267 0.794982 0.798119 0
320 21467.6 0.735029 0.787725 0.791503 0
340 21134.2 0.729297 0.782128 0.786425 0
360 20865.9 0.724652 0.777723 0.78244 0
380 20645.3 0.720811 0.774198 0.779252 0
400 20460.6 0.717577 0.771334 0.776661 0
420 20303.2 0.714813 0.76898 0.774529 0
440 20167.1 0.712413 0.767022 0.772753 0
460 20047.7 0.710299 0.765379 0.771261 0
480 19941.3 0.708412 0.763987 0.769996 0
500 1984

In [22]:
train_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_train, R_test)))
test_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_test, test_R_test)))
valid_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_valid, valid_R_test)))
mae_tr, mae_v, mae_t = sess.run([train_mae, valid_mae, test_mae])
print(mae_tr)
print(mae_v)
print(mae_t)

0.499064
0.561465
0.565209


In [23]:
movie_rating = pickle.load(open('../data/'+folder+'/movie_rating.pkl', 'rb'))
movie_rating_test = pickle.load(open('../data/'+folder+'/movie_rating_test.pkl', 'rb'))

In [24]:
_U, _P, _result = sess.run([U,P, result])

In [25]:
import numpy as np
_p = np.transpose(_P)

In [26]:
total_not_in = 0
vv_predict = []
vv_y = []
largest_num = 10
_RMSE_list = []
for largest_num in range(0,200, 3): 
    for m in range(0, max_movie+1):
        if (m not in movie_rating or len(movie_rating[m]) <=largest_num) and m in movie_rating_test:
            #print('not in', m)
            #print(movie_rating_test[m])
            for pair in movie_rating_test[m]:
                movie_n = m
                user_n = pair[0]
                vv_predict.append(np.dot(_U[user_n], _p[movie_n]))
                vv_y.append(pair[1])
            total_not_in += len(movie_rating_test[m])

    _RMSE = np.sqrt(np.mean(np.square(np.subtract(vv_predict, vv_y))))
    _RMSE_list.append((largest_num, _RMSE))
    

In [27]:
total_not_in

23916

In [28]:
np.sqrt(np.mean(np.square(np.subtract(vv_predict, vv_y))))

0.87263121712097491

In [29]:
_P[0][0], _P[1][0], _P[2][0], _P[3][0]

(-0.88079619, 1.7541928, 2.1570361, -3.3290749)

In [30]:
_p[0]

array([-0.88079619,  1.75419283,  2.15703607, -3.32907486,  0.43785998], dtype=float32)

In [31]:
_RMSE_list

[(0, 3.8032438158988953),
 (3, 3.8032438158988953),
 (6, 3.8032438158988957),
 (9, 3.8032438158988953),
 (12, 2.9162890541691171),
 (15, 2.2713762384744887),
 (18, 1.8895044658938838),
 (21, 1.672133805068615),
 (24, 1.5720587660783232),
 (27, 1.4827465363242989),
 (30, 1.4305947019386607),
 (33, 1.3754494515190032),
 (36, 1.3400730710855555),
 (39, 1.3021144395383892),
 (42, 1.2657075130252828),
 (45, 1.2294875932708178),
 (48, 1.1970791457934333),
 (51, 1.172476271682438),
 (54, 1.1511278975697987),
 (57, 1.1337542923158619),
 (60, 1.1138781681230741),
 (63, 1.0949396026507185),
 (66, 1.0783737151904815),
 (69, 1.0653020057905134),
 (72, 1.0508332384108132),
 (75, 1.0390618349988823),
 (78, 1.0272608975674333),
 (81, 1.0173894384944819),
 (84, 1.0090089112533789),
 (87, 0.99975163003232959),
 (90, 0.99001291879616082),
 (93, 0.98126866430302329),
 (96, 0.97212233046435514),
 (99, 0.96283689479719325),
 (102, 0.95455102700687267),
 (105, 0.94653573689480153),
 (108, 0.9388310336388050

In [32]:
import csv
with open('MF_RMSE.csv','w') as f:
    f_csv = csv.writer(f)
    headers = ['num, RMSE']
    f_csv.writerow(headers)
    for p in _RMSE_list:
        row = [p[0], p[1]]
        f_csv.writerow(row)